# Model Evaluation

In [ ]:
import torch
from torch.distributions import kl_divergence, Bernoulli
from tqdm import tqdm
from environments import PairedComparison
from model import GRU
from baselines import gini, Guessing, VariationalProbitRegression, VariationalEqualWeighting, VariationalFirstCue, VariationalBestCue, VariationalFirstDiscriminatingCue, StrategySelection, FeedForward
from GPyOpt.methods import BayesianOptimization
import numpy as np
import math

## Generate Data Files

In [ ]:
for condition in [1, 2, 3, 4]:
    if condition == 1:
        inputs_a, inputs_b, targets, predictions, time_elapsed = torch.load('data/humans_ranking.pth')
        output_file = 'data/exp1.csv'

    elif condition == 2:
        inputs_a, inputs_b, targets, predictions, time_elapsed = torch.load('data/humans_direction.pth')
        output_file = 'data/exp2.csv'

    elif condition == 3:
        inputs_a, inputs_b, targets, predictions, time_elapsed = torch.load('data/humans_none.pth')
        output_file = 'data/exp3.csv'
        
    elif condition == 4:
        inputs_a, inputs_b, targets, predictions, time_elapsed, weights, direction1, direction2, ranking = torch.load('data/humans_2features.pth')
        output_file = 'data/exp4.csv'

    features = inputs_a - inputs_b
    num_participants = features.shape[0]
    num_tasks = features.shape[2]
    num_steps = features.shape[1]

    print(time_elapsed.shape)
    print(num_participants, num_tasks, num_steps)

    records = np.empty((num_participants * num_tasks * num_steps, 10))
    c = 0
    for participant in range(num_participants):
        for task in range(num_tasks):
            for step in range(num_steps):
                records[c, 0] = participant
                records[c, 1] = task
                records[c, 2] = step
                records[c, 3] = features[participant, step, task, 0]
                records[c, 4] = features[participant, step, task, 1]
                if condition < 4:
                    records[c, 5] = features[participant, step, task, 2]
                    records[c, 6] = features[participant, step, task, 3]                   
                records[c, 7] = predictions[participant, step, task, 0]
                records[c, 8] = targets[participant, step, task, 0]
                records[c, 9] = time_elapsed[participant, step, task, 0]
                c += 1


    np.savetxt(output_file,
        records,
        fmt=",".join(["%d"] + ["%d"] + ["%d"] + ["%.2f"] + ["%.2f"] + ["%.2f"] + ["%.2f"] + ["%d"] + ["%d"] + ["%.2f"]),
        header="participant,task,step,x0,x1,x2,x3,choice,target,time",
        comments='')

## Evaluate Baselines

In [ ]:
num_experiments = 100
num_episodes = 30
sequence_length = 10

for condition in [1, 2, 3, 4]:
    if condition == 1:
        save_path = 'data/baselines_ranking.pth'
        models = [VariationalProbitRegression, VariationalEqualWeighting, VariationalFirstCue]
        ranking = True
        direction = False
        num_features = 4
    elif condition == 2:
        save_path = 'data/baselines_direction.pth'
        models = [VariationalProbitRegression, VariationalEqualWeighting, VariationalBestCue]
        ranking = False
        direction = True
        num_features = 4
    elif condition == 3:
        save_path = 'data/baselines_none.pth'
        models = [VariationalProbitRegression, VariationalEqualWeighting, VariationalBestCue]
        ranking = False
        direction = False
        num_features = 4
    elif condition == 4:
        save_path = 'data/baselines_none_2cues.pth'
        models = [VariationalProbitRegression, VariationalEqualWeighting, VariationalBestCue]
        ranking = False
        direction = False
        num_features = 2

    data_loader = PairedComparison(num_features, ranking=ranking, direction=direction, dichotomized=False)

    map_performance = torch.zeros(len(models), num_experiments, num_episodes, sequence_length)
    avg_performance = torch.zeros(len(models), num_experiments, num_episodes, sequence_length)
    gini_coefficients = torch.zeros(1, num_experiments, num_episodes, sequence_length)

    for j, model_class in enumerate(models):
        for k in tqdm(range(num_experiments)):
            for i in range(num_episodes):
                inputs, targets, _, _ = data_loader.get_batch(1, sequence_length)
                model = model_class(data_loader.num_inputs, noise=data_loader.sigma)
                predictive_distribution = model.forward(inputs, targets)

                prediction = (predictive_distribution.probs > 0.5).float()
                map_performance[j, k, i] = (prediction == targets).squeeze()
                avg_performance[j, k, i] = ((1 - targets) * (1 - predictive_distribution.probs) + targets * predictive_distribution.probs).squeeze()

                if j == 0:
                    for t in range(sequence_length):
                        gini_coefficients[j, k, i, t] = gini(torch.abs(model.weights[t].t()).squeeze().detach().cpu().numpy())

    print(map_performance.mean(1).mean(1))
    print(avg_performance.mean(1).mean(1))
    torch.save([map_performance, avg_performance, gini_coefficients], save_path)

## Evaluate BMI

In [ ]:
num_experiments = 1000
num_episodes = 30
sequence_length = 10

for condition in [1, 2, 3, 4]:
    if condition == 1:
        ranking = True
        direction = False

        file_names = [
            'trained_models/ranking_1_0.pth',
            'trained_models/ranking_2_0.pth',
            'trained_models/ranking_3_0.pth',
            'trained_models/ranking_4_0.pth',
            'trained_models/ranking_5_0.pth',
            'trained_models/ranking_6_0.pth',
            'trained_models/ranking_pretrained_0.pth'
            ]

        save_path = 'data/bmli_ranking.pth'
        num_features = 4

    elif condition == 2:
        ranking = False
        direction = True

        file_names = [
            'trained_models/direction_1_0.pth',
            'trained_models/direction_2_0.pth',
            'trained_models/direction_3_0.pth',
            'trained_models/direction_4_0.pth',
            'trained_models/direction_5_0.pth',
            'trained_models/direction_6_0.pth',
            'trained_models/direction_pretrained_0.pth'
            ]

        save_path = 'data/bmli_direction.pth'
        num_features = 4

    elif condition == 3:
        ranking = False
        direction = False

        file_names = [
            'trained_models/none_1_0.pth',
            'trained_models/none_2_0.pth',
            'trained_models/none_3_0.pth',
            'trained_models/none_4_0.pth',
            'trained_models/none_5_0.pth',
            'trained_models/none_6_0.pth',
            'trained_models/none_pretrained_0.pth'
            ]

        save_path = 'data/bmli_none.pth'
        num_features = 4
        
    elif condition == 4:
        ranking = False
        direction = False

        file_names = [
            'trained_models/none_2cues_1_full_0.pth',
            'trained_models/none_2cues_2_full_0.pth',
            'trained_models/none_2cues_3_full_0.pth',
            'trained_models/none_2cues_4_full_0.pth',
            'trained_models/none_2cues_5_full_0.pth',
            'trained_models/none_2cues_6_full_0.pth',
            'trained_models/none_2cues_pretrained_0.pth'
        ]

        save_path = 'data/bmli_none_2cues.pth'
        num_features = 2
        

    data_loader = PairedComparison(num_features, ranking=ranking, direction=direction, dichotomized=False)

    gini_coefficients = torch.zeros(len(file_names), num_experiments, num_episodes, sequence_length)
    map_performance = torch.zeros(len(file_names), num_experiments, num_episodes, sequence_length)
    avg_performance = torch.zeros(len(file_names), num_experiments, num_episodes, sequence_length)

    for m, file_name in enumerate(file_names):
        model = GRU(data_loader.num_inputs, data_loader.num_targets, 128)

        params, _ = torch.load(file_name, map_location='cpu')
        model.load_state_dict(params)

        for k in tqdm(range(num_experiments)):
            for i in range(num_episodes):
                inputs, targets, _, _ = data_loader.get_batch(1, sequence_length)
                predictive_distribution, weights, variances = model(inputs, targets)

                map_performance[m, k, i] = ((predictive_distribution.probs > 0.5).float() == targets).squeeze().detach()
                avg_performance[m, k, i] = ((1 - targets) * (1 - predictive_distribution.probs) + targets * predictive_distribution.probs).squeeze().detach()

                for j in range(sequence_length):
                    gini_coefficients[m, k, i, j] = gini(torch.abs(weights[j].t()).squeeze().detach().cpu().numpy())

    print(map_performance.mean(1).mean(1))
    torch.save([map_performance, avg_performance, gini_coefficients], save_path)

## KL Divergences

In [ ]:
num_experiments = 100
num_episodes = 30
sequence_length = 10

for condition in [0, 1, 2]:
    if condition == 0:
        save_path = 'data/kl_ranking.pth'
        ranking = True
        direction = False
        models = [VariationalEqualWeighting, VariationalFirstCue]
        file_names = [
            'trained_models/ranking_2_0.pth',
            'trained_models/ranking_pretrained_0.pth'
            ]

    elif condition == 1:
        save_path = 'data/kl_direction.pth'
        ranking = False
        direction = True
        models = [VariationalEqualWeighting, VariationalBestCue]
        file_names = [
            'trained_models/direction_2_0.pth',
            'trained_models/direction_pretrained_0.pth'
            ]

    elif condition == 2:
        save_path = 'data/kl_none.pth'
        ranking = False
        direction = False
        models = [VariationalEqualWeighting, VariationalBestCue]
        file_names = [
            'trained_models/none_2_0.pth',
            'trained_models/none_pretrained_0.pth'
            ]



    data_loader = PairedComparison(4, ranking=ranking, direction=direction, dichotomized=False)
    kl_divergences = torch.zeros(3, len(models), num_experiments, num_episodes, sequence_length)

    bmi = GRU(4, 1, 128)
    params_bmi, _ = torch.load(file_names[0], map_location='cpu')
    bmi.load_state_dict(params_bmi)

    mi = GRU(4, 1, 128)
    params_mi, _ = torch.load(file_names[1], map_location='cpu')
    mi.load_state_dict(params_mi)

    for k in tqdm(range(num_experiments)):
        for i in range(num_episodes):
            inputs, targets, _, _ = data_loader.get_batch(1, sequence_length)

            io = VariationalProbitRegression(data_loader.num_inputs, noise=data_loader.sigma)
            io_pd = io.forward(inputs, targets)

            bmi_pd, _, _ = bmi(inputs, targets)
            mi_pd, _, _ = mi(inputs, targets)

            for j, model_class in enumerate(models):
                model = model_class(data_loader.num_inputs, noise=data_loader.sigma)
                predictive_distribution = model.forward(inputs, targets)
                kl_divergences[0, j, k, i] = kl_divergence(predictive_distribution, bmi_pd).squeeze()
                kl_divergences[1, j, k, i] = kl_divergence(predictive_distribution, mi_pd).squeeze()
                kl_divergences[2, j, k, i] = kl_divergence(predictive_distribution, io_pd).squeeze()


    torch.save([kl_divergences], save_path)

## Baseline Model Fits

In [ ]:
for condition in [1, 2, 3, 4]:
    if condition == 1:
        load_path = "data/humans_ranking.pth"
        num_features = 4
        inputs_a, inputs_b, targets, predictions, _ = torch.load(load_path)
    elif condition == 2:
        load_path = "data/humans_direction.pth"
        num_features = 4
        inputs_a, inputs_b, targets, predictions, _ = torch.load(load_path)
    elif condition == 3:
        load_path = "data/humans_none.pth"
        num_features = 4
        inputs_a, inputs_b, targets, predictions, _ = torch.load(load_path)
    elif condition == 4:
        load_path = "data/humans_2features.pth"
        num_features = 2
        inputs_a, inputs_b, targets, predictions, time_elapsed, _, _, _, _ = torch.load(load_path)
        
    save_path = 'data/logprobs' + str(condition) + '_fitted.pth'

    if condition == 1:
        models = [Guessing, VariationalProbitRegression, VariationalEqualWeighting, VariationalFirstCue]
    else:
        models = [Guessing, VariationalProbitRegression, VariationalEqualWeighting, VariationalBestCue]

    logprobs = torch.zeros(inputs_a.shape[0], len(models))
    params = torch.zeros(inputs_a.shape[0], len(models))
    for participant in tqdm(range(inputs_a.shape[0])):
        for m, model_class in enumerate(models):
            def f(sigma):
                current_logprobs = 0
                for task in range(inputs_a.shape[2]):
                    model = model_class(num_features, noise=sigma[0, 0])
                    participant_inputs = inputs_a[participant, :, [task]] - inputs_b[participant, :, [task]]
                    participant_targets = targets[participant, :, [task]]
                    predictive_distribution = model.forward(participant_inputs, participant_targets)
                    current_logprobs = current_logprobs + predictive_distribution.log_prob(predictions[participant, :, [task]]).sum().item()

                return -current_logprobs

            myBopt = BayesianOptimization(f=f, domain=[{'name': 'var_1', 'type': 'continuous', 'domain': (0.01, 10)}])
            myBopt.run_optimization(max_iter=100)
            print(myBopt.x_opt)
            print(myBopt.fx_opt)
            params[participant, m] = myBopt.x_opt[0]
            logprobs[participant, m] = -myBopt.fx_opt

    print(logprobs / 2.303)
    print(torch.argmax(logprobs, -1))

    torch.save([logprobs, params], save_path)

## Strategy Selection Model Fits

In [ ]:
for condition in [1, 2, 3, 4]:
    if condition == 1:
        load_path = "data/humans_ranking.pth"
        ranking = True
        num_features = 4
        inputs_a, inputs_b, targets, predictions, _ = torch.load(load_path)
    elif condition == 2:
        load_path = "data/humans_direction.pth"
        ranking = False
        num_features = 4
        inputs_a, inputs_b, targets, predictions, _ = torch.load(load_path)
    elif condition == 3:
        load_path = "data/humans_none.pth"
        ranking = False
        num_features = 4
        inputs_a, inputs_b, targets, predictions, _ = torch.load(load_path)
    elif condition == 4:
        load_path = "data/humans_2features.pth"
        ranking = False
        num_features = 2
        inputs_a, inputs_b, targets, predictions, time_elapsed, _, _, _, _ = torch.load(load_path)

    save_path = 'data/logprobs' + str(condition) + '_selection_fitted.pth'

    models = [StrategySelection]

    logprobs = torch.zeros(inputs_a.shape[0], len(models))
    params = torch.zeros(inputs_a.shape[0], len(models))
    for participant in tqdm(range(inputs_a.shape[0])):
        for m, model_class in enumerate(models):
            def f(sigma):
                current_logprobs = 0
                for task in range(inputs_a.shape[2]):
                    model = model_class(num_features, ranking=ranking, noise=sigma[0, 0])
                    participant_inputs = inputs_a[participant, :, [task]] - inputs_b[participant, :, [task]]
                    participant_targets = targets[participant, :, [task]]
                    predictive_distribution = model.forward(participant_inputs, participant_targets)
                    current_logprobs = current_logprobs + predictive_distribution.log_prob(predictions[participant, :, [task]]).sum().item()

                return -current_logprobs

            myBopt = BayesianOptimization(f=f, domain=[{'name': 'var_1', 'type': 'continuous', 'domain': (0.01, 10)}])
            myBopt.run_optimization(max_iter=100)
            print(myBopt.x_opt)
            print(myBopt.fx_opt)
            params[participant, m] = myBopt.x_opt[0]
            logprobs[participant, m] = -myBopt.fx_opt

    print(logprobs / 2.303)
    print(params)

    torch.save([logprobs, params], save_path)

## Feedforward Network Model Fits

In [ ]:
for condition in [1, 2, 3, 4]:
    if condition == 1:
        load_path = "data/humans_ranking.pth"
        model_class = FeedForward
        num_features = 4
        inputs_a, inputs_b, targets, predictions, _ = torch.load(load_path)
    elif condition == 2:
        load_path = "data/humans_direction.pth"
        model_class = FeedForward
        num_features = 4
        inputs_a, inputs_b, targets, predictions, _ = torch.load(load_path)
    elif condition == 3:
        load_path = "data/humans_none.pth"
        model_class = FeedForward
        num_features = 4
        inputs_a, inputs_b, targets, predictions, _ = torch.load(load_path)
    elif condition == 4:
        load_path = "data/humans_2features.pth"
        model_class = FeedForward
        num_features = 2
        inputs_a, inputs_b, targets, predictions, time_elapsed, _, _, _, _ = torch.load(load_path)
        

    save_path = 'data/logprobs' + str(condition) + '_feedforward_fitted.pth'

    logprobs = torch.zeros(inputs_a.shape[0])
    params = torch.zeros(inputs_a.shape[0], 2)
    for participant in tqdm(range(inputs_a.shape[0])):
        def f(params):
            current_logprobs = 0
            for task in range(inputs_a.shape[2]):
                model = model_class(num_features, noise=params[0, 0], learning_rate=params[0, 1])
                participant_inputs = inputs_a[participant, :, [task]] - inputs_b[participant, :, [task]]
                participant_targets = targets[participant, :, [task]]
                predictive_distribution = model.forward(participant_inputs, participant_targets)
                current_logprobs = current_logprobs + predictive_distribution.log_prob(predictions[participant, :, [task]]).sum().item()

            return -current_logprobs

        myBopt = BayesianOptimization(f=f, domain=[{'name': 'var_1', 'type': 'continuous', 'domain': (0.01, 10)}, {'name': 'var_2', 'type': 'continuous', 'domain': (0.0, 0.1)}])
        myBopt.run_optimization(max_iter=100)
        print(torch.from_numpy(myBopt.x_opt))
        print(myBopt.fx_opt)
        logprobs[participant] = -myBopt.fx_opt
        params[participant, :] = torch.from_numpy(myBopt.x_opt)

    print(logprobs / 2.303)
    print(params)

    torch.save([logprobs, params], save_path)

## BMI Model Fits

In [ ]:
num_samples = 100
for condition in [1, 2, 3, 4]:
    if condition == 1:
        load_path = "data/humans_ranking.pth"
        num_features = 4
        inputs_a, inputs_b, targets, predictions, _ = torch.load(load_path)
    elif condition == 2:
        load_path = "data/humans_direction.pth"
        num_features = 4
        inputs_a, inputs_b, targets, predictions, _ = torch.load(load_path)
    elif condition == 3:
        load_path = "data/humans_none.pth"
        num_features = 4
        inputs_a, inputs_b, targets, predictions, _ = torch.load(load_path)
    elif condition == 4:
        load_path = "data/humans_2features.pth"
        num_features = 2
        inputs_a, inputs_b, targets, predictions, time_elapsed, _, _, _, _ = torch.load(load_path)
        
    save_path = 'data/logprobs' + str(condition) + '_bmli.pth'

    

    if condition == 1:
        model_paths = [
            'trained_models/ranking_1_0.pth',
            'trained_models/ranking_2_0.pth',
            'trained_models/ranking_3_0.pth',
            'trained_models/ranking_4_0.pth',
            'trained_models/ranking_5_0.pth',
            'trained_models/ranking_6_0.pth',
            'trained_models/ranking_pretrained_0.pth'
            ]
    elif condition == 2:
        model_paths = [
            'trained_models/direction_1_0.pth',
            'trained_models/direction_2_0.pth',
            'trained_models/direction_3_0.pth',
            'trained_models/direction_4_0.pth',
            'trained_models/direction_5_0.pth',
            'trained_models/direction_6_0.pth',
            'trained_models/direction_pretrained_0.pth'
            ]

    elif condition == 3:
        model_paths = [
            'trained_models/none_1_0.pth',
            'trained_models/none_2_0.pth',
            'trained_models/none_3_0.pth',
            'trained_models/none_4_0.pth',
            'trained_models/none_5_0.pth',
            'trained_models/none_6_0.pth',
            'trained_models/none_pretrained_0.pth'
            ]
    elif condition == 4:
        model_paths = [
            'trained_models/none_2cues_1_full_0.pth',
            'trained_models/none_2cues_2_full_0.pth',
            'trained_models/none_2cues_3_full_0.pth',
            'trained_models/none_2cues_4_full_0.pth',
            'trained_models/none_2cues_5_full_0.pth',
            'trained_models/none_2cues_6_full_0.pth',
            'trained_models/none_2cues_pretrained_0.pth'
        ]

    logprobs = torch.zeros(inputs_a.shape[0], len(model_paths))
    # for each participant
    with torch.no_grad():
        for participant in tqdm(range(inputs_a.shape[0])):
            # for each model
            for m, model_path in enumerate(model_paths):
                model = GRU(num_features, 1, 128)

                params, _ = torch.load(model_path, map_location='cpu')
                model.load_state_dict(params)

                participant_inputs = inputs_a[participant] - inputs_b[participant]
                participant_targets = targets[participant]

                avg_probs = 0
                for sample in range(num_samples):
                    predictive_distribution, _, _ = model(participant_inputs, participant_targets)
                    avg_probs += predictive_distribution.probs

                avg_predictive_distribution = Bernoulli(avg_probs / num_samples)
                logprobs[participant, m] = avg_predictive_distribution.log_prob(predictions[participant]).sum()

        print(logprobs / 2.303)
        print(torch.argmax(logprobs, -1))
        torch.save([logprobs], save_path)

## Evidence

In [ ]:
for condition in [1, 2, 3, 4]:
    load_path = 'data/logprobs' + str(condition) + '_fitted.pth'
    logprobs = torch.load(load_path)[0]
    logprobs[:, 1:] = -0.5 * math.log(300) + logprobs[:, 1:]

    # add strategy selection
    load_path = 'data/logprobs' + str(condition) + '_selection_fitted.pth'
    logprobs_selection = torch.load(load_path)[0]
    logprobs_selection = -0.5 * math.log(300) + logprobs_selection
    logprobs = torch.cat([logprobs, logprobs_selection], dim=-1)

    # add feedforward
    logprobs_feedforward = torch.load('data/logprobs' + str(condition) + '_feedforward_fitted.pth')[0]
    logprobs_feedforward = -0.5 * 2 * math.log(300) + logprobs_feedforward
    logprobs = torch.cat([logprobs, logprobs_feedforward.unsqueeze(1)], dim=-1)

    if condition >= 3:
        logprobs_meta = torch.load('data/logprobs' + str(condition) + '_bmli.pth')[0]
        logprobs_meta = torch.cat([logprobs[:, [0]], logprobs_meta], dim=-1)
        best_logprobs, best_index = torch.max(logprobs_meta, dim=-1)
        bic = -0.5 * 1 * math.log(300) + best_logprobs
        logprobs = torch.cat([logprobs, bic.unsqueeze(1)], dim=-1)
    torch.save(logprobs.detach(), 'data/evidence' + str(condition) + '.pth')

## Log-likelihoods for each time-step

In [ ]:
num_samples = 100
for condition in [1, 2, 3, 4]:
    if condition == 1:
        load_path = "data/humans_ranking.pth"
        _, baseline_parameters = torch.load('data/logprobs1_fitted.pth')
        bmi_logprobs = torch.load('data/logprobs' + str(condition) + '_bmli.pth')
        model_paths = [
            'trained_models/ranking_1_0.pth',
            'trained_models/ranking_2_0.pth',
            'trained_models/ranking_3_0.pth',
            'trained_models/ranking_4_0.pth',
            'trained_models/ranking_5_0.pth',
            'trained_models/ranking_6_0.pth',
            'trained_models/ranking_pretrained_0.pth'
            ]
        num_features = 4
        inputs_a, inputs_b, targets, predictions, _ = torch.load(load_path)

    elif condition == 2:
        load_path = "data/humans_direction.pth"
        _, baseline_parameters = torch.load('data/logprobs2_fitted.pth')
        bmi_logprobs = torch.load('data/logprobs' + str(condition) + '_bmli.pth')
        model_paths = [
            'trained_models/direction_1_0.pth',
            'trained_models/direction_2_0.pth',
            'trained_models/direction_3_0.pth',
            'trained_models/direction_4_0.pth',
            'trained_models/direction_5_0.pth',
            'trained_models/direction_6_0.pth',
            'trained_models/direction_pretrained_0.pth'
            ]
        num_features = 4
        inputs_a, inputs_b, targets, predictions, _ = torch.load(load_path)

    elif condition == 3:
        load_path = "data/humans_none.pth"
        _, baseline_parameters = torch.load('data/logprobs3_fitted.pth')
        bmi_logprobs = torch.load('data/logprobs' + str(condition) + '_bmli.pth')
        model_paths = [
            'trained_models/none_1_0.pth',
            'trained_models/none_2_0.pth',
            'trained_models/none_3_0.pth',
            'trained_models/none_4_0.pth',
            'trained_models/none_5_0.pth',
            'trained_models/none_6_0.pth',
            'trained_models/none_pretrained_0.pth'
            ]
        num_features = 4
        inputs_a, inputs_b, targets, predictions, _ = torch.load(load_path)
        
    elif condition == 4:
        load_path = "data/humans_2features.pth"
        _, baseline_parameters = torch.load('data/logprobs4_fitted.pth')
        bmi_logprobs = torch.load('data/logprobs' + str(condition) + '_bmli.pth')
        model_paths = [
            'trained_models/none_2cues_1_full_0.pth',
            'trained_models/none_2cues_2_full_0.pth',
            'trained_models/none_2cues_3_full_0.pth',
            'trained_models/none_2cues_4_full_0.pth',
            'trained_models/none_2cues_5_full_0.pth',
            'trained_models/none_2cues_6_full_0.pth',
            'trained_models/none_2cues_pretrained_0.pth'
        ]
        num_features = 2
        inputs_a, inputs_b, targets, predictions, time_elapsed, _, _, _, _ = torch.load(load_path)

    bmi_index = torch.argmax(bmi_logprobs[0], -1)
    save_path = 'data/logprobs' + str(condition) + '_time.pth'

    if condition == 1:
        models = [Guessing, VariationalProbitRegression, VariationalEqualWeighting, VariationalFirstCue]
    else:
        models = [Guessing, VariationalProbitRegression, VariationalEqualWeighting, VariationalBestCue]

    logprobs = torch.zeros(len(models) + 1, inputs_a.shape[0], inputs_a.shape[1], inputs_a.shape[2])
    for m, model_class in enumerate(models):
        for participant in tqdm(range(inputs_a.shape[0])):
            for task in range(inputs_a.shape[2]):
                model = model_class(num_features, noise=baseline_parameters[participant, m].item())
                participant_inputs = inputs_a[participant, :, [task]] - inputs_b[participant, :, [task]]
                participant_targets = targets[participant, :, [task]]
                predictive_distribution = model.forward(participant_inputs, participant_targets)
                logprobs[m, participant, :, task] = -predictive_distribution.log_prob(predictions[participant, :, [task]]).squeeze()

    with torch.no_grad():
        for participant in tqdm(range(inputs_a.shape[0])):
            model = GRU(4, 1, 128)

            params, _ = torch.load(model_paths[bmi_index[participant]], map_location='cpu')
            model.load_state_dict(params)

            participant_inputs = inputs_a[participant] - inputs_b[participant]
            participant_targets = targets[participant]

            avg_probs = 0
            for sample in range(num_samples):
                predictive_distribution, _, _ = model(participant_inputs, participant_targets)
                avg_probs += predictive_distribution.probs

            avg_predictive_distribution = Bernoulli(avg_probs / num_samples)
            logprobs[-1, participant] = -avg_predictive_distribution.log_prob(predictions[participant]).squeeze()

    print(logprobs / 2.303) #

    torch.save([logprobs], save_path)

## Judgements about Ranking and Direction

In [ ]:
inputs_a, inputs_b, targets, predictions, time_elapsed, weights, direction1, direction2, ranking = torch.load('data/humans_2features.pth')

# ranking
participant_ranking = ranking.squeeze() # 1 means attribut 0 is more important
actual_ranking = (weights[:,0,:,0] > weights[:,0,:,1]).float()
print((participant_ranking == actual_ranking).float().mean())

participant_direction1 = (direction1.squeeze()).float() # 1 means positve
actual_direction1 = (weights[:,0,:,0] > 0).float()
print((participant_direction1 == actual_direction1).float().mean())

participant_direction2 = (direction2.squeeze()).float() # 1 means positve
actual_direction2 = (weights[:,0,:,1] > 0).float()
print((participant_direction2 == actual_direction2).float().mean())

_, baseline_parameters = torch.load('data/logprobs4_fitted.pth')
baseline_parameters = baseline_parameters[:, 1]
print(baseline_parameters.shape)

io_ranking = torch.zeros(inputs_a.shape[0], inputs_a.shape[2], 1)
io_direction1 = torch.zeros(inputs_a.shape[0], inputs_a.shape[2], 1)
io_direction2 = torch.zeros(inputs_a.shape[0], inputs_a.shape[2], 1)
io_mean = torch.zeros(inputs_a.shape[0], inputs_a.shape[2], 2)
io_std = torch.zeros(inputs_a.shape[0], inputs_a.shape[2], 2)
for participant in tqdm(range(inputs_a.shape[0])):
    for task in range(inputs_a.shape[2]):
        model = VariationalProbitRegression(2, noise=baseline_parameters[participant].item())
        participant_inputs = inputs_a[participant, :, [task]] - inputs_b[participant, :, [task]]
        participant_targets = targets[participant, :, [task]]
        predictive_distribution = model.forward(participant_inputs, participant_targets)
        mean = model.weights[:, :, 0]
        std = torch.exp(model.log_sigmas[:, :, 0])

        io_mean[participant, task] = mean[-1, :]
        io_std[participant, task] = std[-1, :]
        mean_difference = mean[-1, 0] - mean[-1, 1]
        std_difference = (std[-1, 0].pow(2) + std[-1, 1].pow(2)).sqrt()
        io_ranking[participant, task] = Normal(torch.zeros([]), torch.ones([])).cdf(mean_difference / std_difference)
        io_direction1[participant, task] = Normal(torch.zeros([]), torch.ones([])).cdf(mean[-1, 0] / std[-1, 0])
        io_direction2[participant, task] = Normal(torch.zeros([]), torch.ones([])).cdf(mean[-1, 1] / std[-1, 1])


model_paths = [
    'trained_models/none_2cues_1_full_0.pth',
    'trained_models/none_2cues_2_full_0.pth',
    'trained_models/none_2cues_3_full_0.pth',
    'trained_models/none_2cues_4_full_0.pth',
    'trained_models/none_2cues_5_full_0.pth',
    'trained_models/none_2cues_6_full_0.pth',
    'trained_models/none_2cues_pretrained_0.pth'
]

num_samples = 100
logprobs = torch.load('data/logprobs4_bmli.pth')[0]
best_model = torch.argmax(logprobs, -1)

bmi_ranking = torch.zeros(inputs_a.shape[0], inputs_a.shape[2], num_samples)
bmi_direction1 = torch.zeros(inputs_a.shape[0], inputs_a.shape[2], num_samples)
bmi_direction2 = torch.zeros(inputs_a.shape[0], inputs_a.shape[2], num_samples)
bmi_mean = torch.zeros(inputs_a.shape[0], inputs_a.shape[2], num_samples, 2)
bmi_std = torch.zeros(inputs_a.shape[0], inputs_a.shape[2], num_samples, 2)
# for each participant
with torch.no_grad():
    for participant in range(inputs_a.shape[0]):
        # for each model
        print(model_paths[best_model[participant].item()])
        model_path = model_paths[best_model[participant].item()]
        model = GRU(2, 1, 128)

        params, _ = torch.load(model_path, map_location='cpu')
        model.load_state_dict(params)

        participant_inputs = inputs_a[participant] - inputs_b[participant]
        participant_targets = targets[participant]

        for sample in range(num_samples):
            predictive_distribution, mean, std = model(participant_inputs, participant_targets)

            bmi_mean[participant, :, sample, :] = mean[-1, :]
            bmi_std[participant, :, sample, :] = std[-1, :]
            mean_difference = mean[-1, :, 0] - mean[-1, :, 1]
            std_difference = (std[-1, :, 0].pow(2) + std[-1, :, 1].pow(2)).sqrt()
            bmi_ranking[participant, :, sample] = Normal(torch.zeros([]), torch.ones([])).cdf(mean_difference / std_difference) #(mean[-1, :, 0] > mean[-1, :, 1]).float()
            bmi_direction1[participant, :, sample] = Normal(torch.zeros([]), torch.ones([])).cdf(mean[-1, :, 0] / std[-1, :, 0])  #(mean[-1, :, 0] > 0).float() #
            bmi_direction2[participant, :, sample] = Normal(torch.zeros([]), torch.ones([])).cdf(mean[-1, :, 1] / std[-1, :, 1]) #(mean[-1, :, 1] > 0).float()
            print(mean.shape)

torch.save([actual_ranking, actual_direction1, actual_direction2,
    participant_ranking, participant_direction1, participant_direction2,
    io_ranking, io_direction1, io_direction2, io_mean, io_std,
    bmi_ranking, bmi_direction1, bmi_direction2, bmi_mean, bmi_std],
    'data/judgements.pth')

## Model Fits for each task

### Baselines

In [ ]:
for condition in [1, 2, 3]:
    if condition == 1:
        load_path = "data/humans_ranking.pth"
    elif condition == 2:
        load_path = "data/humans_direction.pth"
    elif condition == 3:
        load_path = "data/humans_none.pth"

    save_path = 'data/logprobs' + str(condition) + '_tasks_fitted.pth'

    inputs_a, inputs_b, targets, predictions, _ = torch.load(load_path)

    if condition == 1:
        models = [Guessing, VariationalProbitRegression, VariationalEqualWeighting, VariationalFirstCue]
    else:
        models = [Guessing, VariationalProbitRegression, VariationalEqualWeighting, VariationalBestCue]


    logprobs = torch.zeros(inputs_a.shape[2], inputs_a.shape[0], len(models))
    params = torch.zeros(inputs_a.shape[2], inputs_a.shape[0], len(models))
    for task in tqdm(range(inputs_a.shape[2])):
        for m, model_class in enumerate(models):
            for participant in range(inputs_a.shape[0]):
                def f(sigma):
                    model = model_class(4, noise=sigma[0, 0])
                    participant_inputs = inputs_a[participant, :, [task]] - inputs_b[participant, :, [task]]
                    participant_targets = targets[participant, :, [task]]
                    predictive_distribution = model.forward(participant_inputs, participant_targets)
                    return -predictive_distribution.log_prob(predictions[participant, :, [task]]).sum().item()

                myBopt = BayesianOptimization(f=f, domain=[{'name': 'var_1', 'type': 'continuous', 'domain': (0.01, 10)}])
                myBopt.run_optimization(max_iter=100)
                params[task, participant, m] = myBopt.x_opt[0]
                logprobs[task, participant, m] = -myBopt.fx_opt
            print(logprobs[task, :, m].sum())


    print(logprobs / 2.303) #
    print(torch.argmax(logprobs, -1))

    torch.save([logprobs, params], save_path)

### Strategy Selection

In [ ]:
for condition in [1, 2, 3]:
    if condition == 1:
        load_path = "data/humans_ranking.pth"
        ranking = True
    elif condition == 2:
        load_path = "data/humans_direction.pth"
        ranking = False
    elif condition == 3:
        load_path = "data/humans_none.pth"
        ranking = False

    save_path = 'data/logprobs' + str(condition) + '_tasks_selection_fitted.pth'

    inputs_a, inputs_b, targets, predictions, _ = torch.load(load_path)

    models = [StrategySelection]

    logprobs = torch.zeros(inputs_a.shape[2], inputs_a.shape[0], len(models))
    params = torch.zeros(inputs_a.shape[2], inputs_a.shape[0], len(models))
    for task in tqdm(range(inputs_a.shape[2])):
        for m, model_class in enumerate(models):
            for participant in range(inputs_a.shape[0]):
                def f(sigma):
                    model = model_class(4, noise=sigma[0, 0], ranking=ranking)
                    participant_inputs = inputs_a[participant, :, [task]] - inputs_b[participant, :, [task]]
                    participant_targets = targets[participant, :, [task]]
                    predictive_distribution = model.forward(participant_inputs, participant_targets)
                    return -predictive_distribution.log_prob(predictions[participant, :, [task]]).sum().item()

                myBopt = BayesianOptimization(f=f, domain=[{'name': 'var_1', 'type': 'continuous', 'domain': (0.01, 10)}])
                myBopt.run_optimization(max_iter=100)
                params[task, participant, m] = myBopt.x_opt[0]
                logprobs[task, participant, m] = -myBopt.fx_opt
            print(logprobs[task, :, m].sum())


    print(logprobs / 2.303) #
    print(torch.argmax(logprobs, -1))

    torch.save([logprobs, params], save_path)

### Feedforward Network

In [ ]:
for condition in [1, 2, 3]:
    if condition == 1:
        load_path = "data/humans_ranking.pth"
        ranking = True
    elif condition == 2:
        load_path = "data/humans_direction.pth"
        ranking = False
    elif condition == 3:
        load_path = "data/humans_none.pth"
        ranking = False

    save_path = 'data/logprobs' + str(condition) + '_tasks_feedforward_fitted.pth'

    inputs_a, inputs_b, targets, predictions, _ = torch.load(load_path)

    learning_rates = [0.0, 1/256, 1/128, 1/64, 1/32, 1/16]

    logprobs = torch.zeros(inputs_a.shape[2], inputs_a.shape[0])
    params = torch.zeros(inputs_a.shape[2], inputs_a.shape[0], 2)
    for task in tqdm(range(inputs_a.shape[2])):
        for participant in range(inputs_a.shape[0]):
            def f(params):
                model = FeedForward(4,  noise=params[0, 0], learning_rate=params[0, 1])
                participant_inputs = inputs_a[participant, :, [task]] - inputs_b[participant, :, [task]]
                participant_targets = targets[participant, :, [task]]
                predictive_distribution = model.forward(participant_inputs, participant_targets)
                return -predictive_distribution.log_prob(predictions[participant, :, [task]]).sum().item()

            myBopt = BayesianOptimization(f=f, domain=[{'name': 'var_1', 'type': 'continuous', 'domain': (0.01, 10)}, {'name': 'var_2', 'type': 'continuous', 'domain': (0.0, 0.1)}])
            myBopt.run_optimization(max_iter=100)
            params[task, participant, :] = torch.from_numpy(myBopt.x_opt)
            logprobs[task, participant] = -myBopt.fx_opt
        print(logprobs[task, :].sum())

    print(logprobs / 2.303) #
    torch.save([logprobs, params], save_path)

## Power Analysis

In [ ]:
num_steps = 10
num_tasks = 10000

kl = torch.zeros(2, num_tasks, num_steps)

for k, dichotomized in enumerate([True, False]):
    data_loader = PairedComparison(4, direction=False, dichotomized=dichotomized, ranking=True)
    for i in tqdm(range(num_tasks)):
        if dichotomized:
            model1 = VariationalFirstDiscriminatingCue(data_loader.num_inputs)
        else:
            model1 = VariationalFirstCue(data_loader.num_inputs)
        model2 = VariationalProbitRegression(data_loader.num_inputs)
        inputs, targets, _, _ = data_loader.get_batch(1, num_steps)

        predictive_distribution1 = model1.forward(inputs, targets)
        predictive_distribution2 = model2.forward(inputs, targets)
        kl[k, i, :] =  kl_divergence(predictive_distribution1, predictive_distribution2).squeeze()

torch.save(kl, 'data/power_analysis.pth')
print(kl.mean(1))
print(kl.mean(1).sum(-1))

## Strategy Selection Model

In [ ]:
num_experiments = 100
num_episodes = 30
sequence_length = 10

for condition in [0, 1, 2, 3]:
    if condition == 0:
        save_path = 'data/additional_baselines_ranking.pth'
        models = [StrategySelection] 
        ranking = True
        direction = False
        num_features = 4
    elif condition == 1:
        save_path = 'data/additional_baselines_direction.pth'
        models = [StrategySelection] 
        ranking = False
        direction = True
        num_features = 4
    elif condition == 2:
        save_path = 'data/additional_baselines_none.pth'
        models = [StrategySelection]
        ranking = False
        direction = False
        num_features = 4
    elif condition == 3:
        save_path = 'data/additional_baselines_none_2features.pth'
        models = [StrategySelection]
        ranking = False
        direction = False
        num_features = 2

    data_loader = PairedComparison(num_features, ranking=ranking, direction=direction, dichotomized=False)

    map_performance = torch.zeros(len(models), num_experiments, num_episodes, sequence_length)
    avg_performance = torch.zeros(len(models), num_experiments, num_episodes, sequence_length)
    selected_models = torch.zeros(num_experiments, num_episodes, sequence_length, 3)

    for j, model_class in enumerate(models):
        for k in tqdm(range(num_experiments)):
            for i in range(num_episodes):
                inputs, targets, _, _ = data_loader.get_batch(1, sequence_length)
                model = model_class(data_loader.num_inputs, noise=data_loader.sigma, ranking=ranking)
                predictive_distribution = model.forward(inputs, targets)
                selected_models[k, i] = model.selected_model

                prediction = (predictive_distribution.probs > 0.5).float()
                map_performance[j, k, i] = (prediction == targets).squeeze()
                avg_performance[j, k, i] = ((1 - targets) * (1 - predictive_distribution.probs) + targets * predictive_distribution.probs).squeeze()

    print(map_performance.mean(1).mean(1))
    print(avg_performance.mean(1).mean(1))
    torch.save([map_performance, avg_performance, selected_models], save_path)

## Feedforward Network

In [ ]:
num_experiments = 100
num_episodes = 30
sequence_length = 10

for condition in [0, 1, 2, 3]:
    if condition == 0:
        save_path = 'data/feedforward_baselines_ranking.pth'
        ranking = True
        direction = False
        num_features = 4 
    elif condition == 1:
        save_path = 'data/feedforward_baselines_direction.pth'
        ranking = False
        direction = True
        num_features = 4 
    elif condition == 2:
        save_path = 'data/feedforward_baselines_none.pth'
        ranking = False
        direction = False
        num_features = 4 
    elif condition == 3:
        save_path = 'data/feedforward_baselines_none_2features.pth'
        ranking = False
        direction = False
        num_features = 2

    data_loader = PairedComparison(num_features, ranking=ranking, direction=direction, dichotomized=False)

    learning_rates = [0.0, 1/256, 1/128, 1/64, 1/32, 1/16]
    map_performance = torch.zeros(len(learning_rates), num_experiments, num_episodes, sequence_length)
    avg_performance = torch.zeros(len(learning_rates), num_experiments, num_episodes, sequence_length)
    gini_coefficients = torch.zeros(len(learning_rates), num_experiments, num_episodes, sequence_length)

    for j, learning_rate in enumerate(learning_rates):
        for k in tqdm(range(num_experiments)):
            for i in range(num_episodes):
                inputs, targets, _, _ = data_loader.get_batch(1, sequence_length)
                model = FeedForward(data_loader.num_inputs, learning_rate=learning_rate)
                predictive_distribution = model.forward(inputs, targets)
                prediction = (predictive_distribution.probs > 0.5).float()
                map_performance[j, k, i] = (prediction == targets).squeeze()
                avg_performance[j, k, i] = ((1 - targets) * (1 - predictive_distribution.probs) + targets * predictive_distribution.probs).squeeze()

                for t in range(sequence_length):
                    gini_coefficients[j, k, i, t] = gini(torch.abs(model.means[t].t()).squeeze().detach().cpu().numpy())

    print(map_performance.mean(1).mean(1))
    print(avg_performance.mean(1).mean(1))
    torch.save([map_performance, avg_performance, gini_coefficients], save_path)